Tests our implementation of smoothie

In [3]:
import sys
sys.path.append("..")

from src.smoothie import Smoothie
import numpy as np

In [13]:
# Build distribution of LFs
n_samples = 1000
n_voters = 3
n_dim = 10

# Construct groundtruth theta "accuracies"
theta = np.random.random(n_voters)*50 # all positive for now. Larger theta = less noisy LFs.
sigma_diag = np.zeros(n_voters*n_dim)
for i in range(len(sigma_diag)):
    prompt_idx = int(i / n_dim)
    sigma_diag[i] = 1/(2 * theta[prompt_idx])
sigma = np.diag(sigma_diag)
mu = np.zeros(n_voters * n_dim)


lambda_arr = []
true_y = []
while len(lambda_arr) < n_samples:
    diff = np.random.multivariate_normal(mu, sigma)
    y = np.random.random(n_dim)
    
    y_repeated = np.tile(y, reps= n_voters)
    lambdas = (y_repeated + diff).reshape((n_voters, n_dim))
    lambda_arr.append(lambdas)
    true_y.append(y)

lambda_arr = np.array(lambda_arr)
true_y = np.array(true_y)
lambda_arr.shape

(1000, 3, 10)

In [14]:
model = Smoothie(n_voters, n_dim)
model.fit(lambda_arr)
print("True theta: ", theta)
print("Learned theta: ", model.theta)

True theta:  [16.11798931  7.86106545 10.94555468]
Learned theta:  [15.92251198  7.9550514  10.85990429]


In [19]:
# Measure per-sample error
errs = []
for idx in range(len(true_y)):
    y_pred = model.predict(lambda_arr[idx])
    errs.append(np.linalg.norm(y_pred - true_y[idx], 2)**2)
np.mean(errs)

0.1455943349903884